In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import gc
import traceback

# xsuite
import xtrack as xt
import xmask as xm
import xfields as xf
import xpart as xp
import xobjects as xo


# BBStudies
import sys
sys.path.append('/Users/pbelanger/ABPLocal/BBStudies')
sys.path.append('/home/phbelang/abp/BBStudies')
import BBStudies.Tracking.XsuitePlus as xPlus
import BBStudies.Tracking.Progress as pbar
import BBStudies.Tracking.InteractionPoint as inp
import BBStudies.Physics.Detuning as tune
import BBStudies.Plotting.BBPlots as bbplt
import BBStudies.Physics.Base as phys
import BBStudies.Physics.Constants as cst

# JOB imports
import importlib
sys.path.append('../../')
main_002 = importlib.import_module('Jobs.002_user_specific_tasks.main')
main = importlib.import_module('Jobs.003_particle_dist_and_track.main')





# tracked   = xPlus.Tracking_Interface.from_parquet('zfruits/BUNCH_0224_for_2s' ,partition_name='CHUNK',
#                     start_at_turn = 0,
#                     stop_at_turn  = 2000)
# tracked2   = xPlus.Tracking_Interface.from_parquet('zfruits/BUNCH_0224_for_2s' ,partition_name='CHUNK',
#                     start_at_turn = 20000,
#                     stop_at_turn  = 22000)








tracking_path = 'zfruits/BBB_Signature/FULL/'
tracked   = xPlus.Tracking_Interface.from_parquet(tracking_path ,partition_name='CHUNK')

data_path = 'zfruits/BBB_Signature/DATA/'
data      = xPlus.Tracking_Interface.from_parquet(data_path,partition_name='BUNCH')
_cpt      = xPlus.Tracking_Interface.from_parquet(data_path.replace('DATA','CHECKPOINTS'),partition_name='BUNCH')
data._checkpoint = _cpt._checkpoint


In [ ]:
data.data

In [ ]:
tracked.df.groupby('turn').count()

In [ ]:
data.coord

In [ ]:
plt.figure()
plt.plot(tracked.df.groupby('turn').count().particle)



In [ ]:
tracked.df


In [ ]:
tracked.df


---
# Collimator half-opening 
---

In [ ]:
sig_x = []
sig_y = []
for bunch_ID in [p.name.split('=')[1] for p in list(Path(data_path).rglob('BUNCH*/'))]:
    _data = xPlus.Tracking_Interface.from_parquet(data_path,partition_name='BUNCH',partition_ID=bunch_ID)
    sig_x.append(_data.sig_x)
    sig_y.append(_data.sig_y)



In [ ]:
plt.figure()
plt.plot(sig_x)

In [ ]:
plt.figure()
plt.plot(sig_y)

In [ ]:
coll_opening = 5
coll_x = 100*np.mean(sig_x/np.sqrt(_data.nemitt_x)*np.sqrt(3.5e-6))
coll_y = 100*np.mean(sig_y/np.sqrt(_data.nemitt_y)*np.sqrt(3.5e-6))
coll_s = 1e-3#100*np.mean([coll_x,coll_y])

In [ ]:
coll_opening = 5
coll_x = 2.602e-3/2
coll_y = 1.86e-3/2
coll_s = 2.189e-3/2

---
# Computing Intensity
---

In [ ]:


def lost_condition(x_min,y_min,skew_min,x_max,y_max,skew_max):
    # y_fun_skew = lambda _x: np.tan(np.deg2rad(skew_angle))*_x + coll_s/np.cos(np.deg2rad(180-skew_angle))
    # np.abs(y)>y_fun_skew(np.abs(x)))
    return (np.abs(x_min)>coll_x)|(np.abs(y_min)>coll_y)|(np.abs(x_max)>coll_x)|(np.abs(y_max)>coll_y) |(np.abs(skew_max)>coll_s) |(np.abs(skew_min)>coll_s)

Intensity = {}
survived  = {}
for name,group in data.data.groupby('BUNCH'):


    _lost  = lost_condition(group.x_min,group.y_min,group.skew_min,group.x_max,group.y_max,group.skew_max)
    idx_lost     = group.index[_lost]
    idx_survived = group.index[~_lost]


    # New columns
    try:
        group.insert(0,'beyond_coll',False)
        group.insert(0,'lost',False)
    except:
        group.loc[:,'beyond_coll'] = False
        group.loc[:,'lost'] = False



    group.loc[idx_lost,'beyond_coll'] = True
    group.loc[:,'lost'] = group.groupby('particle').beyond_coll.cumsum().astype(bool)

    Intensity[name] = group[~group.lost].groupby('start_at_turn').count().lost
    survived[name]  = group[~group.lost].groupby('start_at_turn').get_group(group.start_at_turn.max()).particle


In [ ]:
data.data

In [ ]:

# Plotting Intensity
plt.figure()
plt.suptitle(f'coll @ {coll_opening:0.0f} sigma')
for key,_intensity in Intensity.items():
    plt.step(_intensity.index,_intensity.values,where='post',label=f'Bunch {key}')
plt.xlabel('turn')
plt.ylabel('Intensity')
# plt.legend()

# Plotting Intensity
plt.figure(figsize=(10,5))
plt.suptitle(f'coll @ {coll_opening:0.0f} sigma')
for key,_intensity in Intensity.items():
    # plt.step(_intensity.index,_intensity.values,where='post',label=f'Bunch {key}')
    plt.plot([key],[100*np.abs(20000-np.min(_intensity.values))/20000],'o',color='C0')
plt.xlabel('Bunch')
plt.ylabel('% of particles lost')
plt.xticks(list(Intensity.keys())[::4])
plt.xlim(200-10,248+10)
plt.legend()







---
# Survival plot
---

In [ ]:
tracked.df[tracked.df.state !=0].groupby('turn').count()



In [ ]:
coord = tracked.coord_sig.set_index('particle')

for bunch in [200,212,220]:
    # 3 subplots
    fig,ax = plt.subplots(1,4,figsize=(15,5))
    fig.suptitle(f'Bunch {bunch}')
    for _ax,xy in zip(ax,[('x_sig','px_sig'),('y_sig','py_sig'),('x_sig','y_sig'),('zeta_sig','pzeta_sig')]):
        plt.sca(_ax)
        #group = data.data.groupby('BUNCH').get_group(bunch)
        _x,_y = xy

        plt.plot(coord.loc[:,_x],coord.loc[:,_y],'.',color='C2',alpha=0.1,label = 'survived')
        plt.xlabel(_x)
        plt.ylabel(_y)
        plt.axis('equal')

In [ ]:
coord = tracked.coord_sig.set_index('particle')

for bunch in [200,212,220]:
    # 3 subplots
    fig,ax = plt.subplots(1,4,figsize=(15,5))
    fig.suptitle(f'Bunch {bunch}')
    for _ax,xy in zip(ax,[('x_sig','px_sig'),('y_sig','py_sig'),('x_sig','y_sig'),('zeta_sig','pzeta_sig')]):
        plt.sca(_ax)
        #group = data.data.groupby('BUNCH').get_group(bunch)
        _x,_y = xy

        plt.plot(coord.loc[:,_x],coord.loc[:,_y],'.',color='C2',alpha=0.1,label = 'survived')
        plt.xlabel(_x)
        plt.ylabel(_y)
        plt.axis('equal')

In [ ]:
cst.LHC_F_REV

In [ ]:
tracked.df.groupby('turn').count()

In [ ]:
coord = tracked.coord_sig.set_index('particle')

for bunch in [200,212,220]:
    # 3 subplots
    fig,ax = plt.subplots(1,4,figsize=(15,5))
    fig.suptitle(f'Bunch {bunch}')
    for _ax,xy in zip(ax,[('x_sig','px_sig'),('y_sig','py_sig'),('x_sig','y_sig'),('zeta_sig','pzeta_sig')]):
        plt.sca(_ax)
        #group = data.data.groupby('BUNCH').get_group(bunch)
        _x,_y = xy
        _surv = survived[bunch]
        plt.plot(coord.loc[coord.index.isin(_surv),_x],coord.loc[coord.index.isin(_surv),_y],'.',color='C2',alpha=0.1,label = 'survived')
        plt.plot(coord.loc[~coord.index.isin(_surv),_x],coord.loc[~coord.index.isin(_surv),_y],'.',color='C3',alpha=0.1,label = 'lost')
        plt.legend()
        plt.xlabel(_x)
        plt.ylabel(_y)
        plt.axis('equal')

In [ ]:
tracked.df

In [ ]:
coord = tracked.coord.set_index('particle')

for bunch in [200,212,220]:
    # 3 subplots
    fig,ax = plt.subplots(1,4,figsize=(15,5))
    fig.suptitle(f'Bunch {bunch}')
    for _ax,xy in zip(ax,[('x','px'),('y','py'),('x','y'),('zeta','pzeta')]):
        plt.sca(_ax)
        #group = data.data.groupby('BUNCH').get_group(bunch)
        _x,_y = xy
        _surv = survived[bunch]
        plt.plot(coord.loc[coord.index.isin(_surv),_x],coord.loc[coord.index.isin(_surv),_y],'.',color='C2',alpha=0.5,label = 'survived')
        plt.plot(coord.loc[~coord.index.isin(_surv),_x],coord.loc[~coord.index.isin(_surv),_y],'.',color='C3',alpha=0.5,label = 'lost')
        plt.legend()
        plt.xlabel(_x)
        plt.ylabel(_y)
        # plt.axis('equal')

In [ ]:
tracked.coord_sig

In [ ]:

# for (name,df),(name2,calc) in zip(tracked.df.groupby('particle'),calculations.calculations.groupby('particle')):
    # assert name==name2, "Problem with the particle ID"
    # if name in [10,20,30]:

worst_particles = calculations.data.groupby('particle').apply(lambda part: np.max(part.x_max)-np.min(part.x_max)).sort_values(ascending=False).index
# for part_ID in worst_particles[1:20]:#np.arange(10,20):
for part_ID in np.arange(10,20):
    df = tracked.df_sig.groupby('particle').get_group(part_ID)
    calc = calculations.calculations_sig.groupby('particle').get_group(part_ID)

    plt.figure()
    plt.plot(df.turn,df.x_sig,'-')
    plt.step(list(calc.start_at_turn) + [list(calc.stop_at_turn)[-1]],list(calc.x_sig_min) + [np.nan],'-',where='post')
    plt.step(list(calc.start_at_turn) + [list(calc.stop_at_turn)[-1]],list(calc.x_sig_max) + [np.nan],'-',where='post',color='C1')
    # plt.axhline(0.005,color='k')
    # plt.axhline(-0.005,color='k')
    # else:
    #     continue
    # plt.ylim([-0.007,0.007])

In [ ]:

# for (name,df),(name2,calc) in zip(tracked.df.groupby('particle'),calculations.calculations.groupby('particle')):
    # assert name==name2, "Problem with the particle ID"
    # if name in [10,20,30]:

worst_particles = calculations.data.groupby('particle').apply(lambda part: np.max(part.x_max)-np.min(part.x_max)).sort_values(ascending=False).index
for part_ID in worst_particles[1:20]:#np.arange(10,20):
    df = tracked.df.groupby('particle').get_group(part_ID)
    calc = calculations.data.groupby('particle').get_group(part_ID)

    plt.figure()
    plt.plot(df.turn,df.x,'-')
    plt.step(list(calc.start_at_turn) + [list(calc.stop_at_turn)[-1]],list(calc.x_min) + [np.nan],'-',where='post')
    plt.step(list(calc.start_at_turn) + [list(calc.stop_at_turn)[-1]],list(calc.x_max) + [np.nan],'-',where='post',color='C1')
    plt.axhline(0.005,color='k')
    plt.axhline(-0.005,color='k')
    # else:
    #     continue
    plt.ylim([-0.007,0.007])

In [ ]:

# for (name,df),(name2,calc) in zip(tracked.df.groupby('particle'),calculations.calculations.groupby('particle')):
    # assert name==name2, "Problem with the particle ID"
    # if name in [10,20,30]:

# worst_particles = calculations.calculations.groupby('particle').apply(lambda part: np.max(part.x_max)-np.min(part.x_max)).sort_values(ascending=True).index
for part_ID in np.arange(10,20):
    df = tracked.df.groupby('particle').get_group(part_ID)
    calc = calculations.data.groupby('particle').get_group(part_ID)

    plt.figure()
    plt.plot(df.turn,df.x,'-')
    plt.step(list(calc.start_at_turn) + [list(calc.stop_at_turn)[-1]],list(calc.x_min) + [np.nan],'-',where='post')
    plt.step(list(calc.start_at_turn) + [list(calc.stop_at_turn)[-1]],list(calc.x_max) + [np.nan],'-',where='post',color='C1')
    plt.axhline(0.005,color='k')
    plt.axhline(-0.005,color='k')
    # else:
    #     continue
    plt.ylim([-0.007,0.007])

In [ ]:
cst.LHC_F_REV*30*60

In [ ]:
calculations.data

In [ ]:
tracked.df

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(tracked.coord_sig.x_sig,tracked.coord_sig.px_sig,'.',alpha=0.1)
plt.axis('square')

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(tracked.coord_sig.x_sig,tracked.coord_sig.y_sig,'.',alpha=0.1)
plt.axis('square')

In [ ]:
calculations

In [ ]:

# for (name,df),(name2,calc) in zip(tracked.df.groupby('particle'),calculations.calculations.groupby('particle')):
    # assert name==name2, "Problem with the particle ID"
    # if name in [10,20,30]:

# worst_particles = calculations.calculations.groupby('particle').apply(lambda part: np.max(part.x_max)-np.min(part.x_max)).sort_values(ascending=True).index
for part_ID in np.arange(10,20):
    df = tracked.df_n.groupby('particle').get_group(part_ID)
    calc = calculations.calculations.groupby('particle').get_group(part_ID)

    plt.figure()
    plt.plot(df.x_n,df.px_n,'-')
    # plt.step(list(calc.start_at_turn) + [list(calc.stop_at_turn)[-1]],list(calc.x_min) + [np.nan],'-',where='post')
    # plt.step(list(calc.start_at_turn) + [list(calc.stop_at_turn)[-1]],list(calc.x_max) + [np.nan],'-',where='post',color='C1')
    # plt.axhline(0.005,color='k')
    # plt.axhline(-0.005,color='k')
    # else:
    #     continue
    # plt.ylim([-0.007,0.007])
    plt.axis('equal')

In [ ]:
tracked.df[tracked.df['turn']>=901]

In [ ]:
import matplotlib.pyplot as plt
test = calculations._calculations
for part in [0,10,20]:
    plt.figure()
    plt.plot(tracked.df.groupby('particle').get_group(part).turn,tracked.df.groupby('particle').get_group(part).x,'-')
    plt.axhline(test.groupby('start_at_turn').get_group(901).iloc[part].x_min,color='k')
    plt.axhline(test.groupby('start_at_turn').get_group(901).iloc[part].x_max,color='k')
    plt.xlim(900,1000)